# Testing and debugging

This is a Notebook providing debug/error tests for the potentials package

## import

In [1]:
import unittest
import tempfile
from pathlib import Path

import potentials
print(potentials.__version__)

0.3.0


In [2]:
potentials.recordmanager.check_styles()

Record styles that passed import:
- Citation
- Potential
- potential_LAMMPS
- potential_LAMMPS_KIM
- Action
- Request
- FAQ
Record styles that failed import:



## tools

In [3]:
class TestTools(unittest.TestCase):
    
    def test_aslist(self):
        from potentials.tools import aslist
        
        invalue = 'string'
        outvalue = ['string']
        self.assertEqual(aslist(invalue), outvalue)
        
        invalue = ['string']
        outvalue = ['string']
        self.assertEqual(aslist(invalue), outvalue)
        
        invalue = ('string','string')
        outvalue = ['string','string']
        self.assertEqual(aslist(invalue), outvalue)
        
    def test_atomic_symbol(self):
        from potentials.tools import atomic_symbol
        
        self.assertEqual(atomic_symbol(74), 'W')
        
        with self.assertRaises(IndexError):
            atomic_symbol(0)
        
        with self.assertRaises(IndexError):
            atomic_symbol(119)

    def test_atomic_number(self):
        from potentials.tools import atomic_number
        
        self.assertEqual(atomic_number('Fe'), 26)
        
        self.assertEqual(atomic_number('Unh'), atomic_number('Sg'))
        
        with self.assertRaises(ValueError):
            atomic_number('Nt')
        
    def test_atomic_mass(self):
        from potentials.tools import atomic_mass
        
        self.assertEqual(atomic_mass(77), 192.217)

        self.assertEqual(atomic_mass('He-9'), 9.043946)

        with self.assertRaises(ValueError):
            atomic_mass(118, prompt=False)

        with self.assertRaises(IndexError):
            atomic_mass(119)

    def test_uber_open_rmode(self):
        from potentials.tools import uber_open_rmode
        
        contents = "This is the contents of my file."

        # Test string
        with uber_open_rmode(contents) as f:
            self.assertEqual(contents, f.read().decode('UTF-8'))
        
        # Create temporary directory
        with tempfile.TemporaryDirectory() as tmpdir:
            
            # Save content to a file
            contentfile = Path(tmpdir, 'content.txt')
            with open(contentfile, 'w') as f:
                f.write(contents)
        
            # Test filename
            with uber_open_rmode(contentfile) as f:
                self.assertEqual(contents, f.read().decode('UTF-8'))

            # Test file object

            with open(contentfile, 'rb') as openf:
                with uber_open_rmode(openf) as f:
                    self.assertEqual(contents, f.read().decode('UTF-8'))
    
    def test_parse_authors(self):
        from potentials.tools import parse_authors
        
        authors = 'Xiang-Guo Li and Chi Chen and Hui Zheng and Yunxing Zuo and Shyue Ping Ong'
        
        out = [{'givenname': 'X.-G.', 'surname': 'Li'},
               {'givenname': 'C.', 'surname': 'Chen'},
               {'givenname': 'H.', 'surname': 'Zheng'},
               {'givenname': 'Y.', 'surname': 'Zuo'},
               {'givenname': 'S.P.', 'surname': 'Ong'}]
        for iny, outy in zip(parse_authors(authors), out):
            self.assertDictEqual(iny, outy)
        
    def test_numderivative(self):
        from potentials.tools import numderivative
        #...
        
            
unittest.main(argv=[''], verbosity=2, exit=False)

test_aslist (__main__.TestTools) ... ok
test_atomic_mass (__main__.TestTools) ... ok
test_atomic_number (__main__.TestTools) ... ok
test_atomic_symbol (__main__.TestTools) ... ok
test_numderivative (__main__.TestTools) ... ok
test_parse_authors (__main__.TestTools) ... ok
test_uber_open_rmode (__main__.TestTools) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.042s

OK


## Load database

In [4]:
potdb = potentials.Database()

## Action

In [5]:
actions, actions_df = potdb.get_actions(return_df=True)
actions_df

,name,date,type,comment,potentials
0,2009-08-12-New-posting-for-C-H-O,2009-08-12,new posting,New posting for C-H-O,[{'id': '2008--Chenoweth-K-van-Duin-A-C-T-Godd...
1,2009-08-18-New-posting-for-Cu-Ag,2009-08-18,new posting,New posting for Cu-Ag,"[{'id': '2009--Wu-H-H-Trinkle-D-R--Cu-Ag', 'ke..."
2,2009-09-30-New-posting-for-Ni-Al,2009-09-30,new posting,New posting for Ni-Al,"[{'id': '2009--Purja-Pun-G-P-Mishin-Y--Ni-Al',..."
3,2009-12-01-New-posting-for-Ti,2009-12-01,new posting,New posting for Ti,"[{'id': '1992--Ackland-G-J--Ti', 'key': '8a634..."
4,2009-12-01-New-posting-for-Ti-Al,2009-12-01,new posting,New posting for Ti-Al,"[{'id': '2003--Zope-R-R-Mishin-Y--Ti-Al', 'key..."
...,...,...,...,...,...
203,2021-05-26 New posting for Zr-Nb,2021-05-26,new posting,New posting for Zr-Nb,"[{'id': '2021--Starikov-S-Smirnova-D--Zr-Nb', ..."
204,2021-07-06 New posting for Fe,2021-07-06,new posting,New posting for Fe,[{'id': '2021--Starikov-S-Smirnova-D-Pradhan-T...
205,2021-07-06 New posting for Fe-H,2021-07-06,new posting,New posting for Fe-H,"[{'id': '2021--Wen-M--Fe-H', 'key': 'a32373ef-..."
206,2021-07-16 New posting for Bi,2021-07-16,new posting,New posting for Bi,[{'id': '2021--Zhou-H-Dickel-D-E-Baskes-M-I-et...


In [6]:
# Print the 10 newest actions
for i in actions_df.sort_values('date', ascending=False).index[:10]:
    actions[i].html(render=True)

In [7]:
json1 = actions[0].model.json()
json2 = actions[0].build_model().json()
json1==json2

True

## Request

In [8]:
requests, requests_df = potdb.get_requests(return_df=True)
requests_df

,name,date,comment,systems
0,2010-01-01-Co-Ni,2010-01-01,alloys for structural applications,"[{'formula': None, 'elements': ['Co', 'Ni']}]"
1,2010-02-01-Mn-Si,2010-02-01,alloys for structural and transportation appli...,"[{'formula': None, 'elements': ['Mn', 'Si']}]"
2,2010-03-01-Zn-O,2010-03-01,None,"[{'formula': 'ZnO', 'elements': ['Zn', 'O']}]"
3,2010-04-01-Hg,2010-04-01,liquid,"[{'formula': None, 'elements': ['Hg']}]"
4,2010-05-01-Mo-S,2010-05-01,None,"[{'formula': 'MoS<sub>2</sub>', 'elements': ['..."
...,...,...,...,...
105,2020-12-28 Al N,2020-12-28,for aluminum nitride,"[{'formula': None, 'elements': ['Al', 'N']}]"
106,2020-12-30 Bi,2020-12-30,None,"[{'formula': None, 'elements': ['Bi']}]"
107,2021-01-25 U Mo Kr,2021-01-25,None,"[{'formula': None, 'elements': ['U', 'Mo', 'Kr..."
108,2021-03-04 Mg Si,2021-03-04,Buckingham potential,"[{'formula': None, 'elements': ['Mg', 'Si']}]"


In [9]:
for i in range(10):
    requests[i].html(render=True)

## FAQ

In [10]:
faqs, faqs_df = potdb.get_faqs(return_df=True)
faqs_df

,name,question,answer
0,faq,What is the purpose of this project?,The purpose of this project is to provide a re...
1,formats,What are the potential formats?,"<b>Due to the range of formats, we are not abl..."
2,graphs,"I've downloaded a file and plotted it, but the...",There can be several reasons for this. One is...
3,lammps,Where can I download the LAMMPS molecular dyna...,"<a href=""http://lammps.sandia.gov"" class=""exte..."
4,manuscript,I have included the NIST repository website ad...,Please send an email to potentials@nist.gov wh...
5,ref,I have downloaded an interatomic potential and...,In addition to the citation information for th...
6,submit,"I have developed a new interatomic potential, ...",Send the potential to potentials@nist.gov with...


In [11]:
for i in range(len(faqs)):
    faqs[i].html(render=True)

## Citation

In [7]:
citations, citations_df = potdb.get_citations(return_df=True, verbose=True)
citations_df

Found 362 matching Citation records in local library
Found 362 matching Citation records in remote library
 - 0 remote records are new


,name,year_authors,year,volume,url,title,publisher,pages,number,month,...,doi,author,abstract,ENTRYTYPE,ID,numpages,note,day,booktitle,address
0,10.1002_adem.200700047,2007--Paduraru-A-Kenoufi-A-Bailey-N-P-Schiotz-J,2007,9,https://doi.org/10.1002%2Fadem.200700047,An Interatomic Potential for Studying CuZr Bul...,Wiley,505--508,6,jun,...,10.1002/adem.200700047,Anca Pǎduraru and Abder Kenoufi and Nicholas P...,Glass forming ability has been found in only a...,article,P_duraru_2007,NaN,NaN,NaN,NaN,NaN
1,10.1002_jcc.23949,2015--Zhou-X-W-Ward-D-K-Foster-M-E,2015,36,https://doi.org/10.1002%2Fjcc.23949,An analytical bond-order potential for carbon,Wiley,1719--1735,23,may,...,10.1002/jcc.23949,X. W. Zhou and D. K. Ward and M. E. Foster,Carbon is the most widely studied material tod...,article,Zhou_2015,NaN,NaN,NaN,NaN,NaN
2,10.1002_jcc.25573,2018--Zhou-X-W-Foster-M-E-Sills-R-B,2018,39,https://doi.org/10.1002%2Fjcc.25573,An Fe-Ni-Cr embedded atom method potential for...,Wiley,2420--2431,29,oct,...,10.1002/jcc.25573,Xiaowang W. Zhou and Michael E. Foster and Rya...,Fe‐Ni‐Cr stainless‐steels are important struct...,article,Zhou_2018,NaN,NaN,NaN,NaN,NaN
3,10.1002_pssb.201147137,2011--Du-Y-A-Lenosky-T-J-Hennig-R-G-et-al,2011,248,https://doi.org/10.1002%2Fpssb.201147137,Energy landscape of silicon tetra-interstitial...,Wiley,2050-2055,9,jun,...,10.1002/pssb.201147137,Yaojun A. Du and Thomas J. Lenosky and Richard...,Mobile single interstitials can grow into exte...,article,Du_2011,NaN,NaN,NaN,NaN,NaN
4,10.1007_s00161-009-0109-1,2009--Guthikonda-V-S-Elliott-R-S,2009,21,https://doi.org/10.1007%2Fs00161-009-0109-1,An effective interaction potential model for t...,Springer Science and Business Media LLC,269--295,4,jul,...,10.1007/s00161-009-0109-1,Venkata Suresh Guthikonda and Ryan S. Elliott,The unusual properties of shape memory alloys ...,article,Guthikonda_2009,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,2018--s-a-etesami-m-i-baskes-m-laradji-et-al--...,2018--Etesami-S-A-Baskes-M-I-Laradji-M-Asadi-E,2018,NaN,NaN,to be published,NaN,NaN,NaN,NaN,...,NaN,Seyed Alireza Etesami and Michael I. Baskes an...,NaN,unpublished,2018--S-A-Etesami-M-I-Baskes-M-Laradji-et-al--...,NaN,2018--S-A-Etesami-M-I-Baskes-M-Laradji-et-al--...,NaN,NaN,NaN
358,2019--m-i-mendelev--cu-zr,2019--Mendelev-M-I,2019,NaN,NaN,to be published,NaN,NaN,NaN,NaN,...,NaN,M.I. Mendelev,NaN,unpublished,2019--M-I-Mendelev--Cu-Zr,NaN,2019--M-I-Mendelev--Cu-Zr,NaN,NaN,NaN
359,2019--m-i-mendelev--fe-ni-cr,2019--Mendelev-M-I,2019,NaN,NaN,to be published,NaN,NaN,NaN,NaN,...,NaN,M.I. Mendelev,NaN,unpublished,2019--M-I-Mendelev--Fe-Ni-Cr,NaN,2019--M-I-Mendelev--Fe-Ni-Cr,NaN,NaN,NaN
360,2021--mendelev-m--al-sm,2021--Mendelev-M,2021,NaN,NaN,to be published,NaN,NaN,NaN,NaN,...,NaN,Mikhail Mendelev,NaN,unpublished,2021--Mendelev-M--Al-Sm,NaN,2021--Mendelev-M--Al-Sm,NaN,NaN,NaN


In [8]:
citations[0].html(render=True)

## Potential

In [9]:
potentials, potentials_df = potdb.get_potentials(return_df=True, verbose=True)
potentials_df

Found 649 matching Potential records in local library
Found 649 matching Potential records in remote library
 - 0 remote records are new


,name,key,id,recorddate,notes,fictional,elements,othername,modelname,citations,implementations
0,potential.1958--Bernardes-N--Ar,24c37060-bdcd-47a9-ba96-33efe0691930,1958--Bernardes-N--Ar,2019-08-08,NaN,0,[Ar],NaN,NaN,"[{'name': '10.1103_physrev.112.1534', 'year_au...",[{'key': '70bc5886-6004-4ef3-bf14-2a1b8e4cd564...
1,potential.1958--Bernardes-N--Kr,e62cbc12-99ba-469b-8d4e-e26ad40cb15a,1958--Bernardes-N--Kr,2019-08-08,NaN,0,[Kr],NaN,NaN,"[{'name': '10.1103_physrev.112.1534', 'year_au...",[{'key': 'be07eb49-d6fd-4677-8a6a-6ee6a764fbac...
2,potential.1958--Bernardes-N--Ne,90c2411b-6b91-41a9-a562-7544e1bea780,1958--Bernardes-N--Ne,2019-08-08,NaN,0,[Ne],NaN,NaN,"[{'name': '10.1103_physrev.112.1534', 'year_au...",[{'key': '72fe01b3-dd55-414f-9183-585eb9c35df3...
3,potential.1958--Bernardes-N--Xe,d6b56970-f7f5-428a-8830-499e8d0e2715,1958--Bernardes-N--Xe,2019-08-08,NaN,0,[Xe],NaN,NaN,"[{'name': '10.1103_physrev.112.1534', 'year_au...",[{'key': '211bf97c-e643-4a51-9c3e-1252c9cd13c5...
4,potential.1959--Girifalco-L-A-Weizer-V-G--Ag,ab71e60e-3f45-4eb4-9c92-7feb98f7e168,1959--Girifalco-L-A-Weizer-V-G--Ag,2020-10-08,NaN,0,[Ag],NaN,NaN,"[{'name': '10.1103_physrev.114.687', 'year_aut...",[{'key': '3778a2c6-ee92-4415-baf6-d4427f2d12f6...
...,...,...,...,...,...,...,...,...,...,...,...
644,potential.2021--Starikov-S-Smirnova-D-Pradhan-...,48617ddd-9e54-40ad-b65b-3ce6ea580075,2021--Starikov-S-Smirnova-D-Pradhan-T-et-al--Fe,2021-07-06,NaN,0,[Fe],NaN,NaN,"[{'name': '10.1103_physrevmaterials.5.063607',...",[{'key': '0b2e2029-5fde-4e2f-bd96-4e8ea04f3c3c...
645,potential.2021--Wang-G-Xu-Y-Qian-P-Su-Y--Au-Rh,ccd7fceb-a257-40df-812b-b1326955bff1,2021--Wang-G-Xu-Y-Qian-P-Su-Y--Au-Rh,2020-08-25,NaN,0,"[Au, Rh]",NaN,NaN,"[{'name': '10.1016_j.commatsci.2020.110002', '...",[{'key': 'f8f52d2e-597d-435a-926a-09f38094b1da...
646,potential.2021--Wen-M--Fe-H,a32373ef-05b0-419e-b346-6ba5c6b36f61,2021--Wen-M--Fe-H,2021-07-06,NaN,0,"[Fe, H]",NaN,NaN,"[{'name': '10.1016_j.commatsci.2021.110640', '...",[{'key': 'd45b6e62-e69e-4c4a-b0f4-8a6f147f19f0...
647,potential.2021--Zhou-H-Dickel-D-E-Baskes-M-I-e...,0aa1b65e-0641-4845-bab6-ca6b0ed589ce,2021--Zhou-H-Dickel-D-E-Baskes-M-I-et-al--Bi,2021-07-16,NaN,0,[Bi],NaN,NaN,"[{'name': '10.1088_1361-651x_ac095c', 'year_au...",[{'key': 'f9f5cd2b-116b-46da-ab68-323810437f0c...


In [10]:
print(potentials[50].html())

<div xmlns="http://www.w3.org/TR/xhtml1/strict"><script src="https://www.ctcms.nist.gov/potentials/site/showFiles.js">//</script><link href="https://www.ctcms.nist.gov/potentials/site/bootstrap/css/bootstrap.min.css" rel="stylesheet"/><link href="https://www.ctcms.nist.gov/potentials/site/bootstrap/css/ie10-viewport-bug-workaround.css" rel="stylesheet"/><div class="panel panel-default"><div class="panel-heading"><h3 class="panel-title">1988--Tersoff-J--C</h3></div><div class="panel-body"><div class="citation"><b>Citation: </b>J. Tersoff (1988), "Empirical Interatomic Potential for Carbon, with Applications to Amorphous Carbon", <i>Physical Review Letters</i>, <b>61(25)</b>, 2879-2882. DOI: <a href="https://doi.org/10.1103/physrevlett.61.2879" class="external">10.1103/physrevlett.61.2879</a>.<br/></div><div class="abstract"><b>Abstract: </b>An empirical interatomic potential is introduced, which gives a convenient and relatively accurate description of the structural properties and ener

In [15]:
potentials[34].html(render=True)

In [16]:
lmppots, lmppots_df = potdb.get_lammps_potentials(return_df=True, verbose=True)
lmppots_df

Found 411 matching potential_LAMMPS records in local library
Found 411 matching potential_LAMMPS records in remote library
 - 0 remote records are new
No KIM potentials added: list of models is empty


,name,id,key,potid,potkey,units,atom_style,allsymbols,pair_style,status,symbols,elements,artifacts,comments,dois
0,1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1,1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1,062d2ba7-3903-40ae-a772-daa471d107c6,1985--Foiles-S-M--Ni-Cu,301f04ce-9082-4542-8590-489300cd19e8,metal,atomic,0.0,eam,active,"[Cu, Ni]","[Cu, Ni]","[{'filename': 'Cu_smf7.eam', 'label': None, 'u...",Potential 1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr...,[10.1103/physrevb.32.7685]
1,1985--Stillinger-F-H--Si--LAMMPS--ipr1,1985--Stillinger-F-H--Si--LAMMPS--ipr1,d085648c-b3ef-4be8-824b-7093fd22770a,1985--Stillinger-F-H-Weber-T-A--Si,edc31ad6-2b9a-455c-9b5f-e888a672ecbd,metal,atomic,0.0,sw,active,[Si],[Si],"[{'filename': 'Si.sw', 'label': None, 'url': '...",Potential 1985--Stillinger-F-H--Si--LAMMPS--ip...,"[10.1103/physrevb.31.5262, 10.1103/physrevb.33..."
2,1986--Foiles-S-M--Ag--LAMMPS--ipr1,1986--Foiles-S-M--Ag--LAMMPS--ipr1,76a265fc-45ff-49d7-8c64-2044f12402f2,1986--Foiles-S-M-Baskes-M-I-Daw-M-S--Ag,672d54f8-9f48-4200-af56-8a7378ebbc4a,metal,atomic,0.0,eam,active,[Ag],[Ag],"[{'filename': 'Ag_u3.eam', 'label': None, 'url...",Potential 1986--Foiles-S-M--Ag--LAMMPS--ipr1 l...,[10.1103/physrevb.33.7983]
3,1986--Foiles-S-M--Ag-Au-Cu-Ni-Pd-Pt--LAMMPS--ipr1,1986--Foiles-S-M--Ag-Au-Cu-Ni-Pd-Pt--LAMMPS--ipr1,c5afa7e8-6b3b-49cd-ad1c-ae3e4329363a,1986--Foiles-S-M-Baskes-M-I-Daw-M-S--Ag-Au-Cu-...,7a1302de-59cf-4efb-900e-cad845b68ee5,metal,atomic,0.0,eam,active,"[Ag, Au, Cu, Ni, Pd, Pt]","[Ag, Au, Cu, Ni, Pd, Pt]","[{'filename': 'Ag_u3.eam', 'label': None, 'url...",Potential 1986--Foiles-S-M--Ag-Au-Cu-Ni-Pd-Pt-...,[10.1103/physrevb.33.7983]
4,1986--Foiles-S-M--Au--LAMMPS--ipr1,1986--Foiles-S-M--Au--LAMMPS--ipr1,c588810a-b96d-4871-bfe2-cff8a5a7c709,1986--Foiles-S-M-Baskes-M-I-Daw-M-S--Au,ffb66faa-319d-4556-8363-dad3959cd553,metal,atomic,0.0,eam,active,[Au],[Au],"[{'filename': 'Au_u3.eam', 'label': None, 'url...",Potential 1986--Foiles-S-M--Au--LAMMPS--ipr1 l...,[10.1103/physrevb.33.7983]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,2021--Starikov-S--Zr-Nb--LAMMPS--ipr1,2021--Starikov-S--Zr-Nb--LAMMPS--ipr1,51eb2d42-0053-44fc-a139-64c4c19571a7,2021--Starikov-S-Smirnova-D--Zr-Nb,fa000d07-cf5d-474f-8407-5fa5b41a79f1,metal,atomic,0.0,adp,active,"[Zr, Nb]","[Zr, Nb]","[{'filename': 'Zr_Nb.2021.adp.txt', 'label': N...",Potential 2021--Starikov-S--Zr-Nb--LAMMPS--ipr...,[10.1016/j.commatsci.2021.110581]
407,2021--Wang-G--Au-Rh--LAMMPS--ipr1,2021--Wang-G--Au-Rh--LAMMPS--ipr1,f8f52d2e-597d-435a-926a-09f38094b1da,2021--Wang-G-Xu-Y-Qian-P-Su-Y--Au-Rh,ccd7fceb-a257-40df-812b-b1326955bff1,metal,atomic,0.0,adp,active,"[Au, Rh]","[Au, Rh]","[{'filename': 'AuRh.adp.txt', 'label': None, '...",Potential 2021--Wang-G--Au-Rh--LAMMPS--ipr1 li...,[10.1016/j.commatsci.2020.110002]
408,2021--Wen-M--Fe-H--LAMMPS--ipr1,2021--Wen-M--Fe-H--LAMMPS--ipr1,d45b6e62-e69e-4c4a-b0f4-8a6f147f19f0,2021--Wen-M--Fe-H,a32373ef-05b0-419e-b346-6ba5c6b36f61,metal,atomic,0.0,eam/fs,active,"[Fe, H]","[Fe, H]","[{'filename': 'FeH_Wen.eam.fs', 'label': None,...",Potential 2021--Wen-M--Fe-H--LAMMPS--ipr1 list...,[10.1016/j.commatsci.2021.110640]
409,2021--Zhou-H--Bi--LAMMPS--ipr1,2021--Zhou-H--Bi--LAMMPS--ipr1,f9f5cd2b-116b-46da-ab68-323810437f0c,2021--Zhou-H-Dickel-D-E-Baskes-M-I-et-al--Bi,0aa1b65e-0641-4845-bab6-ca6b0ed589ce,metal,atomic,0.0,meam,active,[Bi],[Bi],"[{'filename': 'Bi.parameter.meam', 'label': No...",Potential 2021--Zhou-H--Bi--LAMMPS--ipr1 liste...,[10.1088/1361-651x/ac095c]


In [19]:
print(lmppots[0].pair_info())

print "Potential 1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1 listed in the NIST Interatomic Potentials Repository:"
print "https://www.ctcms.nist.gov/potentials/entry/1985--Foiles-S-M--Ni-Cu/1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1.html"
print "Publication(s) related to the potential:"
print "https://doi.org/10.1103/physrevb.32.7685"
print "Parameter file(s) can be downloaded at:"
print "https://www.ctcms.nist.gov/potentials/Download/1985--Foiles-S-M--Ni-Cu/1/Cu_smf7.eam"
print "https://www.ctcms.nist.gov/potentials/Download/1985--Foiles-S-M--Ni-Cu/1/Ni_smf7.eam"
pair_style eam
pair_coeff 1 1 1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1\Cu_smf7.eam
pair_coeff 2 2 1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1\Ni_smf7.eam
mass 1 63.55
mass 2 58.71


